In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Set the root directory of the dataset
root_dir = '/content/drive/MyDrive/Data for test'

# Create a dictionary to map the label names to integers
label_map = {'Sofa': 0, 'Bed': 1, 'Chair': 2}

# Create a list of all the image file paths and their corresponding labels
data = []
for label in label_map.keys():
    path = os.path.join(root_dir, label)
    for filename in os.listdir(path):
        if filename.endswith('.jpg'):
            file_path = os.path.join(path, filename)
            label_id = label_map[label]
            data.append((file_path, label_id))




In [ ]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)

# Create the training and testing directories and copy the images
train_dir = 'train/'
test_dir = 'test/'

for label_id in label_map.values():
    os.makedirs(os.path.join(train_dir, str(label_id)), exist_ok=True)
    os.makedirs(os.path.join(test_dir, str(label_id)), exist_ok=True)

for file_path, label_id in train_data:
    filename = os.path.basename(file_path)
    shutil.copy(file_path, os.path.join(train_dir, str(label_id), filename))

for file_path, label_id in test_data:
    filename = os.path.basename(file_path)
    shutil.copy(file_path, os.path.join(test_dir, str(label_id), filename))

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [5]:


# Define transforms for training and testing datasets
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [6]:
# Load the data
train_data = datasets.ImageFolder('/content/train', transform=train_transforms)
test_data = datasets.ImageFolder('/content/test', transform=test_transforms)
from torch.utils.data import DataLoader
# # Define the data loaders for the training and testing sets
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)


In [32]:
# Define the model architecture
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_data.classes))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Define data loaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Train the model
num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [33]:
# Set the model to train mode
model.train()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

In [34]:
for epoch in range(num_epochs):
    # Training
    model.train()
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        accuracy = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            logits = model(images)
            accuracy += (logits.argmax(1) == labels).float().sum()
        accuracy /= len(test_data)
    print(f"Epoch {epoch+1}, Validation Accuracy: {accuracy.item()*100}%")

# Save the trained model parameters
torch.save(model.state_dict(), 'model.pth')

Epoch 1, Validation Accuracy: 33.33333432674408%
Epoch 2, Validation Accuracy: 40.00000357627869%
Epoch 3, Validation Accuracy: 56.666672229766846%
Epoch 4, Validation Accuracy: 70.00000476837158%
Epoch 5, Validation Accuracy: 78.33333611488342%
Epoch 6, Validation Accuracy: 83.33333730697632%
Epoch 7, Validation Accuracy: 85.00000238418579%
Epoch 8, Validation Accuracy: 85.00000238418579%
Epoch 9, Validation Accuracy: 85.00000238418579%
Epoch 10, Validation Accuracy: 88.33333849906921%
Epoch 11, Validation Accuracy: 88.33333849906921%
Epoch 12, Validation Accuracy: 90.00000357627869%
Epoch 13, Validation Accuracy: 90.00000357627869%
Epoch 14, Validation Accuracy: 88.33333849906921%
Epoch 15, Validation Accuracy: 90.00000357627869%
Epoch 16, Validation Accuracy: 90.00000357627869%
Epoch 17, Validation Accuracy: 90.00000357627869%
Epoch 18, Validation Accuracy: 90.00000357627869%
Epoch 19, Validation Accuracy: 88.33333849906921%
Epoch 20, Validation Accuracy: 88.33333849906921%


In [35]:
import torch
import torchvision.transforms as transforms
from PIL import Image

import torch.nn.functional as F

# Load the saved model checkpoint
checkpoint = torch.load('model.pth')

# Load the model architecture
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

# Load the model state dictionary
model.load_state_dict(torch.load('model.pth'))
#model.load_state_dict(checkpoint['state_dict'])

# Set the model to evaluation mode
model.eval()

# Define the image transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the image
image = Image.open('/content/3 SEATER SOFA OG.JPG')
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)

# Make a prediction
with torch.no_grad():
    logits = model(image_tensor)
    probas = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probas).item()
    
print(predicted_class)


0


In [36]:
# Define the label mapping
label_map = {0:"Sofa", 1:"Bed", 2:"Chair"}

In [37]:
# Get the class label from the predicted label index using the label mapping
predicted_class = label_map[predicted_class]
print(predicted_class)

Sofa


In [44]:
# Load the image
image = Image.open('/content/chair2.png')
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)

# Make a prediction
with torch.no_grad():
    logits = model(image_tensor)
    probas = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probas).item()
    
print(predicted_class)
# Get the class label from the predicted label index using the label mapping
predicted_class = label_map[predicted_class]
print(predicted_class)

RuntimeError: ignored

In [39]:
# Load the image
image = Image.open('/content/bed.jpg')
image_tensor = transform(image)
image_tensor = image_tensor.unsqueeze(0)

# Make a prediction
with torch.no_grad():
    logits = model(image_tensor)
    probas = F.softmax(logits, dim=1)
    predicted_class = torch.argmax(probas).item()
    
print(predicted_class)

# Get the class label from the predicted label index using the label mapping
predicted_class = label_map[predicted_class]
print(predicted_class)

1
Bed


In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [46]:
# Load the data from the root folder of the dataset
data = ImageFolder(root='/content/drive/MyDrive/Data for test', transform=transform)

# Create a DataLoader to load the data in batches during training
batch_size = 16
loader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [47]:
# Define the model architecture
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_data.classes))

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [48]:
# Train the model for a few epochs
num_epochs = 1
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    running_loss = 0.0
    for i, (images, labels) in enumerate(loader):
        optimizer.zero_grad()
        features = model(images)
        logits = model.fc(features)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 9:
            print(f'  batch {i+1}/{len(loader)}: loss {running_loss/10:.3f}')
            running_loss = 0.0

Epoch 1/1


RuntimeError: ignored